In [1]:
import sqlite3 as sq
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle

from tensorflow import keras
from keras.models import Model,Sequential
from keras.layers import Dense,TextVectorization,Embedding, LSTM, Bidirectional

In [2]:
# Connexion à la base de données "db_stack.db" en utilisant la bibliothèque sqlite3
db_stack = sq.connect("db_stack.db")

# Création d'un objet curseur pour exécuter des commandes SQL
cursor = db_stack.cursor()

# Exécution d'une instruction SELECT pour récupérer toutes les données de la table "Stackoverflow"
cursor.execute("SELECT * FROM Stackoverflow")

# Récupération de tous les résultats de la requête précédente et stockage dans la variable "results"
results = cursor.fetchall()


In [3]:
# Création d'un DataFrame à partir des données de la table "Stackoverflow" dans la base de données "db_stack.db"
df = pd.read_sql_query("SELECT * FROM Stackoverflow", db_stack)

# Instanciation de l'objet TextVectorization pour vectoriser les questions de la table
vecteur = TextVectorization(max_tokens=150_000, output_sequence_length=1300, output_mode='int')

# Adaptation du modèle en utilisant les questions de la table
vecteur.adapt(df['Questions'].values)

# Vectorisation des questions en utilisant le modèle adapté
text_vectorise = vecteur(df['Questions'].values) 



In [4]:
text_vectorise

<tf.Tensor: shape=(250, 1300), dtype=int64, numpy=
array([[  4,   2, 116, ...,   0,   0,   0],
       [105, 198, 269, ...,   0,   0,   0],
       [268, 170, 102, ...,   0,   0,   0],
       ...,
       [137, 852, 692, ...,   0,   0,   0],
       [ 68, 597, 513, ...,   0,   0,   0],
       [  9,  41,   2, ...,   0,   0,   0]], dtype=int64)>

In [5]:
# Utilisation de la fonction from_tensor_slices de Tensorflow pour créer un dataset à partir des vecteurs de textes et des étiquettes Toxic
dataset = tf.data.Dataset.from_tensor_slices((text_vectorise, df['Toxic'].values)).cache().shuffle(60000).batch(10).prefetch(8)

# On divise les données en 80% pour l'entrainement, 10% pour la validation et 10% pour les tests
octante = int(len(dataset)*.8)
dix = int(len(dataset)*.1)

# Séparation des données en trois parties : entraînement, validation et test
train,val,test = (dataset.take(octante),dataset.skip(octante).take(dix),dataset.skip(octante+dix).take(dix))


In [6]:
# Création d'un modèle de réseau de neurones séquentiel
model = Sequential()

# Ajout de couches d'embedding, de LSTM bidirectionnel, de couches densément connectées au modèle
model.add(Embedding(150_001, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilation du modèle en spécifiant l'optimiseur Adam et la fonction de perte BinaryCrossentropy
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
             loss=tf.keras.losses.BinaryCrossentropy())

# Entraînement du modèle sur les données d'entraînement avec une validation sur les données de validation
res_train = model.fit(train, epochs=5, validation_data=val)


Epoch 1/5
20/20 [==============================] - 19s 653ms/step - loss: 0.4929 - val_loss: 0.0382
Epoch 2/5
20/20 [==============================] - 12s 593ms/step - loss: 0.4087 - val_loss: 0.4023
Epoch 3/5
20/20 [==============================] - 12s 592ms/step - loss: 0.3747 - val_loss: 0.1937
Epoch 4/5
20/20 [==============================] - 12s 577ms/step - loss: 0.2300 - val_loss: 0.1380
Epoch 5/5
20/20 [==============================] - 12s 580ms/step - loss: 0.1113 - val_loss: 0.0804


In [7]:
# Sauvegarde du model 
model.save('model_sof.h5')

# Sauvegarde du vecteur
pickle.dump({'config': vecteur.get_config(),
             'weights': vecteur.get_weights()}
            , open("monVecteur.pkl", "wb"))